# **Elastic Net Regression**

In [221]:
import pandas as pd

In [222]:
df = pd.read_csv('Laptops.csv')
df.head()

,id,Brand,Model Name,Processor,Operating System,Storage,RAM,Screen Size,Touch_Screen,Price
0,0,HP,15s-fq5007TU,Core i3,Windows 11 Home,512 GB,8 GB,39.62 cm (15.6 Inch),No,"₹38,990"
1,1,HP,15s-fy5003TU,Core i3,Windows 11 Home,512 GB,8 GB,39.62 cm (15.6 Inch),No,"₹37,990"
2,2,Apple,2020 Macbook Air,M1,Mac OS Big Sur,256 GB,8 GB,33.78 cm (13.3 inch),No,"₹70,990"
3,3,Apple,2020 Macbook Air,M1,Mac OS Big Sur,256 GB,8 GB,33.78 cm (13.3 inch),No,"₹70,990"
4,4,Apple,2020 Macbook Air,M1,Mac OS Big Sur,256 GB,8 GB,33.78 cm (13.3 inch),No,"₹70,990"


In [223]:
df.isnull().sum()

id                   0
Brand                0
Model Name           0
Processor            0
Operating System     0
Storage             12
RAM                  0
Screen Size          0
Touch_Screen         0
Price                0
dtype: int64

In [224]:
df['Storage'] = df['Storage'].fillna(value=df['Storage'].mode()[0])

In [225]:
df.isnull().sum()

id                  0
Brand               0
Model Name          0
Processor           0
Operating System    0
Storage             0
RAM                 0
Screen Size         0
Touch_Screen        0
Price               0
dtype: int64

In [226]:
pd.DataFrame(pd.DataFrame(df.groupby('Storage'))[0])

,0
0,1 TB
1,128 GB
2,2 TB
3,256 GB
4,3 TB
5,4 TB
6,512 GB
7,6 TB
8,64 GB


### **Converting Numeric Columns into numeric types**

In [227]:
def convert_storage(storage):
    numeric_part, unit = storage[:-2], storage[-2:]
    numeric_part = int(numeric_part)
    if unit == 'TB':
        numeric_part *= 1024
    return numeric_part

df['Storage'] = df['Storage'].apply(convert_storage)

In [228]:
pd.DataFrame(pd.DataFrame(df.groupby('RAM'))[0])

,0
0,12 GB
1,16 GB
2,18 GB
3,32 GB
4,4 GB
5,64 GB
6,8 GB


In [229]:
pd.DataFrame(pd.DataFrame(df.groupby('Screen Size'))[0]).head()

,0
0,100.63 cm (39.62 cm)
1,17.78 cm (7 Inch)
2,26.67 cm (10.5 inch)
3,29.46 cm (11.6 Inch)
4,29.46 cm (11.6 inch)


In [230]:
df['RAM'] = df['RAM'].astype(str).str.extract('(\d+)').astype(int)
df['Screen Size'] = df['Screen Size'].astype(str).str.extract(r'([\d.]+)').astype(float)
df['Price'] = df['Price'].astype(str).str.replace(r'[^\d]', '', regex=True).astype(int)

In [231]:
pd.DataFrame(df['Screen Size']).head()

,Screen Size
0,39.62
1,39.62
2,33.78
3,33.78
4,33.78


In [232]:
df.drop(columns=['id','Model Name'], inplace=True)
df.head()

,Brand,Processor,Operating System,Storage,RAM,Screen Size,Touch_Screen,Price
0,HP,Core i3,Windows 11 Home,512,8,39.62,No,38990
1,HP,Core i3,Windows 11 Home,512,8,39.62,No,37990
2,Apple,M1,Mac OS Big Sur,256,8,33.78,No,70990
3,Apple,M1,Mac OS Big Sur,256,8,33.78,No,70990
4,Apple,M1,Mac OS Big Sur,256,8,33.78,No,70990


### **Split Train Test data**

In [233]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='Price'),df['Price'],test_size=0.2)

### **Separating columns of different types**

In [234]:
numerical_columns = [3, 4, 5]
ordinal_columns = [0, 6]
nominal_columns = [1, 2]

### **Importing required libararies**

In [235]:
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

In [236]:
impute_num = Pipeline(steps=[
    ('impute_num',SimpleImputer(strategy='mean'))
])

In [237]:
encode_ord = Pipeline(steps=[
    ('impute_cat',SimpleImputer(strategy='most_frequent')),
    ('encode_ord',OrdinalEncoder())
])

In [238]:
encode_nom = Pipeline(steps=[
    ('impute_cat',SimpleImputer(strategy='most_frequent')),
    ('encode_nom',OneHotEncoder(drop='first', handle_unknown='ignore'))
])

In [239]:
preprocessing = ColumnTransformer(transformers=[
    ('impute_num',impute_num,numerical_columns),
    ('encode_ord',encode_ord,ordinal_columns),
    ('encode_nom',encode_nom,nominal_columns)
],remainder='passthrough')

### **Linear Regression Model**

In [240]:
model = LinearRegression()

In [241]:
pipe = Pipeline(steps=[
    ('preprocessor',preprocessing),
    ('model',model)
])

In [242]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_num',
                                                  Pipeline(steps=[('impute_num',
                                                                   SimpleImputer())]),
                                                  [3, 4, 5]),
                                                 ('encode_ord',
                                                  Pipeline(steps=[('impute_cat',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_ord',
                                                                   OrdinalEncoder())]),
                                                  [0, 6]),
                                                 ('encode_nom',
                                                  Pipeline(steps=[('impute_cat',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_nom',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  [1, 2])])),
                ('model', LinearRegression())])

In [243]:

from sklearn.metrics import r2_score

y_pred = pipe.predict(X_test)

LinearRegressionR2score = r2_score(y_test,y_pred)

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [244]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

### **Ridge Model**

In [245]:
ridge_model = Ridge()

In [246]:
ridge_pipe = make_pipeline(preprocessing, ridge_model)

In [247]:
ridge_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_num',
                                                  Pipeline(steps=[('impute_num',
                                                                   SimpleImputer())]),
                                                  [3, 4, 5]),
                                                 ('encode_ord',
                                                  Pipeline(steps=[('impute_cat',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_ord',
                                                                   OrdinalEncoder())]),
                                                  [0, 6]),
                                                 ('encode_nom',
                                                  Pipeline(steps=[('impute_cat',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_nom',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  [1, 2])])),
                ('ridge', Ridge())])

In [248]:
ridge_ypred = ridge_pipe.predict(X_test)

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [249]:
RidgeR2score = r2_score(y_test,y_pred)

### **Lasso Model**

In [250]:
lasso_model = Lasso()

In [251]:
lasso_pipe = make_pipeline(preprocessing, lasso_model)

In [252]:
lasso_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_num',
                                                  Pipeline(steps=[('impute_num',
                                                                   SimpleImputer())]),
                                                  [3, 4, 5]),
                                                 ('encode_ord',
                                                  Pipeline(steps=[('impute_cat',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_ord',
                                                                   OrdinalEncoder())]),
                                                  [0, 6]),
                                                 ('encode_nom',
                                                  Pipeline(steps=[('impute_cat',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_nom',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  [1, 2])])),
                ('lasso', Lasso())])

In [253]:
lasso_ypred = lasso_pipe.predict(X_test)

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [254]:
lasso_r2score = r2_score(y_test, lasso_ypred)

### **ElasticNet Model**

In [255]:
elasticnet_model = ElasticNet()

In [256]:
elasticnet_pipe = make_pipeline(preprocessing, elasticnet_model)

In [257]:
elasticnet_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_num',
                                                  Pipeline(steps=[('impute_num',
                                                                   SimpleImputer())]),
                                                  [3, 4, 5]),
                                                 ('encode_ord',
                                                  Pipeline(steps=[('impute_cat',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_ord',
                                                                   OrdinalEncoder())]),
                                                  [0, 6]),
                                                 ('encode_nom',
                                                  Pipeline(steps=[('impute_cat',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_nom',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  [1, 2])])),
                ('elasticnet', ElasticNet())])

In [258]:
elasticnet_ypred = elasticnet_pipe.predict(X_test)

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [259]:
elastic_r2score = r2_score(y_test, elasticnet_ypred)

### **Result**

In [260]:
print('R2 score from Linear Regression: ', LinearRegressionR2score)
print('R2 score from Ridge Regression: ', RidgeR2score)
print('R2 score from Lasso Regression: ', lasso_r2score)
print('R2 score from ElasticNet Regression: ', elastic_r2score)

R2 score from Linear Regression:  0.40932506508425914
R2 score from Ridge Regression:  0.40932506508425914
R2 score from Lasso Regression:  0.40358073483788004
R2 score from ElasticNet Regression:  0.4452298054791811
